In [1]:
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from datasets import load_dataset
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import pandas as pd
import random
from torch.utils.data import DataLoader
import numpy as np
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

# Set seed for reproducibility
def set_seed_(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed_(42)
set_seed(42)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !pip install transformers==4.50.0

In [3]:
# !pip install datasets

In [4]:
# !pip install scikit-learn

In [5]:
!huggingface-cli login --token {hf_token}

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llm_finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_finetuning`


In [6]:
!nvidia-smi

Fri Dec 26 09:00:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:17:00.0 Off |                    0 |
| N/A   59C    P0             73W /  300W |   79517MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [7]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/workspace/prev/ChainofThought


In [8]:
from datasets import load_dataset

ds = load_dataset("thenlpresearcher/test_data_human_validated_eng_mar")

In [9]:
df_test = ds['test']
df_test

Dataset({
    features: ['sent_written', 'sent_meant', 'gt', 'punct_type'],
    num_rows: 54
})

In [10]:
len(df_test['sent_written'])

54

In [11]:
df_test['sent_written'][0]

'Chanting the choir raised the volume as the celebrant intoned the prayer.'

In [30]:
def get_responses_stereotype_with_zero_zero(sentences):
    def prompt(sentence):
        return f"English: {sentence}\nMarathi Translation (Devanagari) मराठी अनुवादः"
    
    responses = []
    for s in tqdm(sentences):
        responses.append(inference(prompt(s)))
    return responses

def get_responses_stereotype_with_zero_cot(sentences):
    def prompt(sentence):
        return (f"Task: Restore punctuation and translate to Marathi.\n"
                f"Input English: {sentence}\n"
                f"मराठी अनुवाद:")
    
    responses = []
    for s in tqdm(sentences):
        responses.append(inference(prompt(s)))
    return responses

def get_responses_stereotype_with_few_zero(sentences):
    def prompt(sentence):
        return f"""English: These are the components required motor brushes, bearings, and wiring.
                Marathi Translation मराठी अनुवाद: आवश्यक असलेले घटक खालीलप्रमाणे आहेत: मोटार ब्रशेस, बेअरिंग्ज आणि वायरिंग.

                English: As the machine develops the forms we use to record data from past projects will be amended.
                Marathi Translation मराठी अनुवाद: जसजशी यंत्रणा विकसित होईल, तसतसे मागील प्रकल्पांतील डेटा रेकॉर्ड करण्यासाठी आम्ही वापरत असलेले फॉर्म्स सुधारित केले जातील.

                English: {sentence}
                Marathi Translation मराठी अनुवाद:"""
    
    responses = []
    for s in tqdm(sentences):
        responses.append(inference(prompt(s)))
    return responses

def get_responses_stereotype_with_few_cot(sentences):
    def prompt(sentence):
        return f"""Input English: These are the components required motor brushes, bearings, and wiring.
Step 1 (Restoration): These are the components required: motor brushes, bearings, and wiring.
Step 2 (Marathi Translation) मराठी अनुवाद: आवश्यक असलेले घटक खालीलप्रमाणे आहेत: मोटार ब्रशेस, बेअरिंग्ज आणि वायरिंग.
Reasoning: Added a colon to introduce a list of components.

Input English: {sentence}
(Marathi Translation) मराठी अनुवाद:"""
    
    responses = []
    for s in tqdm(sentences):
        # We start the completion at Step 1, the model will likely finish 
        # Step 2 and Reasoning because of the pattern.
        responses.append(inference(prompt(s)))
    return responses

In [53]:
def inference(prompt, max_tokens=128, temperature=0.3):
    # We use a stop sequence logic or just truncate because base models 
    # might keep rambling after the answer is done.
    output = pipe(prompt, 
                  max_new_tokens=max_tokens, 
                  temperature=temperature, 
                  repetition_penalty=1.2,
                  do_sample=True)[0]['generated_text']
    # Return only the new generated part
    return output[len(prompt):].split('\n\n')[0].strip()

In [15]:
import re
import pandas as pd

def classify_stereotype_with_cot(sentences, responses):
    # Initialize lists for answers, reasons, sentences, responses, and answer classifications
    all_answers, all_reasons, all_sentences, all_responses, all_answer_classifications = [], [], [], [], []

    # Process each response in the batch
    for sentence, response in zip(sentences, responses):
        text = response
        text = text.replace('*','')
        all_responses.append(response)
        all_sentences.append(sentence)

        # Define regex patterns for extracting answer and reason
        answer_pattern = r"(?:\*\*Answer\*\*|Answer:)\s*(.*?)(?=\s*(?:Reason:|\Z))"  # non-greedy match between Answer and Reason (or end)
        reason_pattern = r"Reason:\s*(.*)"
        
        # Extract answer and reason using regex
        answer_match = re.search(answer_pattern, text)
        reason_match = re.search(reason_pattern, text)

        # Check if matches are found and extract
        if answer_match and reason_match:
            answer = answer_match.group(1).strip()
            reason = reason_match.group(1).strip()

            # Append to lists
            all_answers.append(answer)
            all_reasons.append(reason)

            # Classify the answer based on keywords
            answer_ = answer.lower()
            if "anti" in answer_:
                answer_classification = 0
            elif "stereotype" in answer_:
                answer_classification = 1
            elif "bias" in answer_:
                answer_classification = 4
            elif "without" in answer_:
                answer_classification = 2
            elif "neutral" in answer_:
                answer_classification = 3
            else:
                answer_classification = -1  # If no classification is found

            all_answer_classifications.append(answer_classification)
        else:
            print(f"Answer or Reason not found for sentence: {sentence}")
            # Handle missing answer/reason case by appending None or placeholders
            all_answers.append(None)
            all_reasons.append(None)
            all_answer_classifications.append(-1)  # Indicating a classification issue

    # Create a DataFrame from the collected data
    df = pd.DataFrame({
        'Sentence': all_sentences,
        'Response': all_responses,
        'Answer': all_answers,
        'Reason': all_reasons,
        'Answer_Classification': all_answer_classifications
    })

    # Save the DataFrame to a CSV (or other formats as needed)
#     df.to_csv("stereotype_classifications.csv", index=False)
    
    return df

In [10]:
# !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [18]:
from transformers import pipeline
device_stereo = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pipe = pipeline(model='sarvamai/sarvam-2b-v0.5', device=device_stereo)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.59it/s]
Device set to use cuda


In [19]:
print(device_stereo)

cuda


In [67]:
get_responses_stereotype_with_zero_zero(['These are the components required motor brushes, bearings, and wiring. मराठी अनुवाद'])

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.82it/s]


['[/INST]']

In [40]:
sentences = list(df_test['sent_written'])
    
responses = get_responses_stereotype_with_zero_cot(sentences)

 89%|█████████████████████████████████████████████████████████████████████████▊         | 48/54 [00:24<00:03,  1.94it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [00:27<00:00,  2.00it/s]


In [41]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/sarvam_zero_cot.csv", index=False)

In [42]:
sentences = list(df_test['sent_written'])
    
responses = get_responses_stereotype_with_zero_zero(sentences)

# for sentence,response in zip(sentences,responses):
#     print(f"Sentence: {sentence}")
#     print(response)
#     print('\n')

100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [00:48<00:00,  1.11it/s]


In [43]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/sarvam_zero_zero.csv", index=False)

In [44]:
sentences = list(df_test['sent_written'])
    
responses = get_responses_stereotype_with_few_cot(sentences)

# for sentence,response in zip(sentences,responses):
#     print(f"Sentence: {sentence}")
#     print(response)
#     print('\n')

100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [00:30<00:00,  1.74it/s]


In [45]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/sarvam_few_cot.csv", index=False)

In [46]:
sentences = list(df_test['sent_written'])
    
responses = get_responses_stereotype_with_few_zero(sentences)

# for sentence,response in zip(sentences,responses):
#     print(f"Sentence: {sentence}")
#     print(response)
#     print('\n')

100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [00:53<00:00,  1.02it/s]


In [47]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/sarvam_few_zero.csv", index=False)

In [48]:
def get_responses_stereotype_with_original(sentences, max_new_tokens=64):
    def prompt(sentence):
        return f"English: {sentence}\nMarathi Translation (Devanagari) मराठी अनुवादः"
    
    responses = []
    for s in tqdm(sentences):
        responses.append(inference(prompt(s)))
    return responses

In [49]:
sentences = list(df_test['sent_meant'])
    
responses = get_responses_stereotype_with_original(sentences)

# for sentence,response in zip(sentences,responses):
#     print(f"Sentence: {sentence}")
#     print(response)
#     print('\n')

100%|███████████████████████████████████████████████████████████████████████████████████| 54/54 [00:50<00:00,  1.07it/s]


In [50]:
df = pd.DataFrame({
        'Sentence': sentences,
        'Response': responses
    })

# Save the DataFrame to a CSV (or other formats as needed)
df.to_csv("responses/sarvam_original.csv", index=False)